In [4]:
import subprocess
import os
import pandas as pd

def ucitaj_sekvencu_iz_fajla(fajl_putanja):
    """
    Učitava sekvencu iz FASTA formata fajla, uklanja zaglavlje i vraća sekvencu kao string.
    """
    with open(fajl_putanja, 'r') as fajl:
        linije = fajl.readlines()
    
    sekvenca = "".join([linija.strip() for linija in linije[1:]])  # Početni red je zaglavlje
    return sekvenca

def generisi_fasta_za_prozor(prozor, indeks):
    """
    Generiše FASTA fajl za svaki prozor.
    """
    fasta_putanja = f"Prozori/prozor_{indeks}.fasta"
    with open(fasta_putanja, 'w') as f:
        f.write(f">prozor_{indeks}\n{prozor}")
    return fasta_putanja

def pokreni_isunstruct_na_fajlu(fajl_putanja):
    """
    Pokreće ISUNSTRUCT na datom FASTA fajlu i vraća parsirane rezultate iz .iul fajla.
    """
    isunstruct_putanja = "/home/user/Desktop/IsUnstruct_2.02/IsUnstruct"
    folder_prozora = "/home/user/Desktop/Istrazivanje-podataka-2/Validacija/ISUNSTRUCT_predikcije/Prozori"

    apsolutna_putanja_fajla = os.path.join(folder_prozora, os.path.basename(fajl_putanja))

    rezultati = subprocess.run(
        [isunstruct_putanja, apsolutna_putanja_fajla],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    if rezultati.stderr:
        print(f"Greska pri pokretanju ISUNSTRUCT: {rezultati.stderr}")

    # Proveravamo .iul fajl
    folder_predikcije = "/home/user/Desktop/Istrazivanje-podataka-2/Validacija/ISUNSTRUCT_predikcije"
    izlazni_fajl = os.path.join(folder_predikcije, os.path.basename(fajl_putanja).replace(".fasta", ".iul"))


    if os.path.exists(izlazni_fajl):
        return parsiraj_isunstruct(izlazni_fajl)
    else:
        print(f"Upozorenje: Ne postoji .iul fajl za {fajl_putanja}")
        return []

def parsiraj_isunstruct(putanja_ius_fajla):
    """
    Parsira izlaz iz ISUNSTRUCT .iul fajla i klasifikuje svaku poziciju u sekvenci.
    """
    rezultati = []
    
    with open(putanja_ius_fajla, "r") as f:
        linije = f.readlines()
    
    for linija in linije:
        linija = linija.strip()
        if linija and not linija.startswith("#"):  # Ignorišemo komentare
            delovi = linija.split()
            if len(delovi) >= 4:
                pozicija = int(delovi[0])
                aminokiselina = delovi[1]
                skor = float(delovi[3])  # Skor je poslednja kolona
                klasifikacija = "uređena" if skor <= 0.5 else "neuređena"
                rezultati.append((pozicija, aminokiselina, skor, klasifikacija))
    
    return rezultati

def generisi_prozore_i_klasifikuj(sekvenca, min_duzina=5, max_duzina=100):
    prozori = []
    oznake_prozora = []
    duzine = []
    pozicije = []
    
    for duzina in range(min_duzina, max_duzina + 1):
        for i in range(len(sekvenca) - duzina + 1):
            podsekvenca = sekvenca[i:i + duzina]
            pozicija = f"{i+1}-{i+duzina}"
            duzine.append(duzina)
            pozicije.append(pozicija)
            
            fasta_putanja = generisi_fasta_za_prozor(podsekvenca, i)
            isunstruct_parsirani_rezultati = pokreni_isunstruct_na_fajlu(fasta_putanja)
            
            podsekvenca_oznake = [result[3] for result in isunstruct_parsirani_rezultati]
            
            if podsekvenca_oznake.count("uređena") > len(podsekvenca_oznake) / 2:
                oznake_prozora.append("uređena")
            else:
                oznake_prozora.append("neuređena")
            
            prozori.append(podsekvenca)
    
    return duzine, pozicije, prozori, oznake_prozora

# Učitavanje sekvence
fajl_putanja = "YP_138520.1 |nucleoprotein.fasta"
sekvenca = ucitaj_sekvencu_iz_fajla(fajl_putanja)

duzine, pozicije, prozori, oznake_prozora = generisi_prozore_i_klasifikuj(sekvenca)

df_sekvence = pd.DataFrame({
    "Duzina": duzine,
    "Pozicija": pozicije,
    "Sekvenca": prozori,
    "ISUNSTRUCT oznaka": oznake_prozora
})

print(df_sekvence.head())
df_sekvence.to_csv("Klasifikovane_sekvence_ISUNSTRUCT_YP_138520.1_nucleoprotein.csv", index=False)


   Duzina Pozicija Sekvenca ISUNSTRUCT oznaka
0       5      1-5    MDKRV         neuređena
1       5      2-6    DKRVR         neuređena
2       5      3-7    KRVRG         neuređena
3       5      4-8    RVRGS         neuređena
4       5      5-9    VRGSW         neuređena
